Concrete and abstract types can be parameterized in Julia.  Suppose we want to create a type represent a point in a two-dimensional space that can have either integer or real coordinates.

# Abstract types and functions

We can define a parameterized abstract type with a type parameter `T` that is a subtype of `Real`.  The concrete types that are subtypes of `AbstractPoint` will have `T` as the types of the fields representing the $x$ and $y$ coordinates.  `Real` is an abstract type that represents floating point values or integers.

In [1]:
abstract type AbstractPoint{T<:Real} end

Now we can define some accessor methods that have a parameterized type for the arguments.  Note that we have to use a `where` clause to indicate that `T` is a type parameter.  For the setters, the type of the value should of course correspond to that of the fields in the strcuture, so `T` as well.

In [2]:
get_x(p::AbstractPoint{T}) where {T} = p.x
set_x!(p::AbstractPoint{T}, val::T) where {T} = p.x = val
get_y(p::AbstractPoint{T}) where {T} = p.y
set_y!(p::AbstractPoint{T}, val::T) where {T} = p.y = val

set_y! (generic function with 1 method)

The `move!` function is parameterized as well, and uses the accessors for maximum reusability.

In [3]:
function move!(p::AbstractPoint{T}, delta_x::T, delta_y::T) where {T}
    set_x!(p, get_x(p) + delta)
    set_y(p, get_y(p) + delta_y)
    return nothing
end    

move! (generic function with 1 method)

Just for sake of the argument, we define a `distance` method for `AbstractPoints` that have `Integer` coordinates as teh Manhattan distance, while for `AbstractFloat` coordinates, we use the Euclidean distance.

In [23]:
function distance(p1::AbstractPoint{T}, p2::AbstractPoint{T}) where {T<:Integer}
    x1, y1 = get_x(p1), get_y(p1)
    x2, y2 = get_x(p2), get_y(p2)
    return abs(x1 - x2) + abs(y1 - y2)
end

distance (generic function with 1 method)

In [24]:
function distance(p1::AbstractPoint{T}, p2::AbstractPoint{T}) where {T<:AbstractFloat}
    x1, y1 = get_x(p1), get_y(p1)
    x2, y2 = get_x(p2), get_y(p2)
    return sqrt((x1 - x2)^2 + (y1 - y2)^2)
end

distance (generic function with 2 methods)

# Concrete type

Now we can define a parameterized concrete type `Point` that is a subtype of `AbstractPoint`.  The type of the `x` and `y` fields is parameterized as `T`, the type parameter of the concrete type and its abstreact supertype.

In [4]:
mutable struct Point{T} <: AbstractPoint{T}
    x::T
    y::T
end

We can now instantiate a `Point` with `Int32` coordiates.  The type of the `x` field as well as the type of the value returned by `get_x` is indeed `Int32`.

In [5]:
p_int32 = Point{Int32}(3, 5)

Point{Int32}(3, 5)

In [16]:
typeof(p_int32.x) == Int32

true

In [17]:
typeof(get_x(p_int32)) == Int32

true

Similarly, we can instantiate a point with `Float32` coordinates, and verify that the fields and values return by the accessor have that type.

In [8]:
p_float32 = Point{Float32}(3.1f0, 4.2f0)

Point{Float32}(3.1f0, 4.2f0)

In [14]:
typeof(p_float32.x) == Float32

true

In [15]:
typeof(get_x(p_float32)) == Float32

true

In [10]:
set_x!(p_int32, Int32(5))

5

Attempting to set the `x` field to a value of the wrong type will result in an error.

In [11]:
set_x!(p_int32, 3.1)

LoadError: MethodError: no method matching set_x!(::Point{Int32}, ::Float64)
[0mClosest candidates are:
[0m  set_x!(::AbstractPoint{T}, [91m::T[39m) where T at In[2]:2

In [12]:
set_x!(p_float32, 7)

LoadError: MethodError: no method matching set_x!(::Point{Float32}, ::Int64)
[0mClosest candidates are:
[0m  set_x!(::AbstractPoint{T}, [91m::T[39m) where T at In[2]:2

In [13]:
p_string = Point{String}("abc", "cde")

LoadError: TypeError: in AbstractPoint, in T, expected T<:Number, got Type{String}

We can verifty that we correct method for the distance function depending on the arguments, Manhattan distance for, e.g., `Int64`, and Euclidean distance for, e.g., `Float64`.

In [25]:
p1_int, p2_int = Point{Int64}(3, 4), Point{Int64}(4, 3)

(Point{Int64}(3, 4), Point{Int64}(4, 3))

In [26]:
distance(p1_int, p2_int)

2

In [27]:
p1_float, p2_float = Point{Float64}(3.0, 4.0), Point{Float64}(4.0, 3.0)

(Point{Float64}(3.0, 4.0), Point{Float64}(4.0, 3.0))

In [28]:
distance(p1_float, p2_float)

1.4142135623730951

Computing the distance between two points of different types will result in an error.

In [29]:
distance(p1_int, p2_float)

LoadError: MethodError: no method matching distance(::Point{Int64}, ::Point{Float64})
[0mClosest candidates are:
[0m  distance(::AbstractPoint{T}, [91m::AbstractPoint{T}[39m) where T<:Integer at In[23]:1
[0m  distance([91m::AbstractPoint{T}[39m, ::AbstractPoint{T}) where T<:AbstractFloat at In[24]:1